# Uso de árboles de decisión para predecir la intención emprendedora media
En esta sección vamos a ver como se comportan los arboles de decisión con nuestro dataset. Vamos a empezar por estos ya que tienen la ventaja de ser algoritmos de los que podemos extraer información facilmente.

In [1]:
import sys
from os.path import join
from pathlib import Path
parent_path = str(Path.cwd().parents[0])

if parent_path not in sys.path:
    sys.path.append(parent_path)


In [2]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from src import manual_preprocessing as mp
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

x_train = pd.read_csv(join(mp.data_path, 'x_train.csv'))
y_train = pd.read_csv(join(mp.data_path, 'y_train.csv'))
x_test = pd.read_csv(join(mp.data_path, 'x_test.csv'))
y_test = pd.read_csv(join(mp.data_path, 'y_test.csv'))
y_cols = y_train.columns

c_cols, n_cols = mp.get_columns_type(x_train)
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                            ('encoder', OrdinalEncoder())])

preprocessor = ColumnTransformer(transformers=[('numerical',  KNNImputer(n_neighbors=2, weights='uniform'), n_cols),
                                                ('categorical', categorical_transformer, c_cols)])
y_imputer = SimpleImputer(strategy='median')
y_train = y_imputer.fit_transform(y_train)
y_test = y_imputer.transform(y_test)

y_train_final = pd.DataFrame(y_train, columns=y_cols)
y_test_final = pd.DataFrame(y_test, columns=y_cols)

x_train_final = preprocessor.fit_transform(x_train)
x_test_final = preprocessor.transform(x_test)

clf = DecisionTreeRegressor(random_state=0).fit(x_train_final, y_train_final['IEMedia'])
y_pred = clf.predict(x_test_final)
y_train_pred = clf.predict(x_train_final)

print(f"R2 score para train {r2_score(y_train_final['IEMedia'], y_train_pred)}")
print(f"R2 score para test {r2_score(y_test_final['IEMedia'], y_pred)}")
print(f"Nivel alcanzado {clf.get_depth()} número de hojas {clf.get_depth()}")


R2 score para train 1.0
R2 score para test 0.4605200296406381
Nivel alcanzado 20 número de hojas 20


Como podemos ver, tenemos un score en train del 100% pero un score de test de un 40%. Esto nos dice que claramente el modelo ha sufrido de sobre-aprendizaje. El árbol de decision de Scikit-Learn no tiene un límite en los niveles que puede crear, asi que empezo a crear tantos niveles hasta que separó completamente el conjunto de datos. Para evitar esto, vamos a ver cual es el número de niveles que tiene este árbol, y empezar a entrenar modelos limitando el número de niveles permitido para el árbol.

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [2, 4, 8, 9, 10, 16],
              'max_leaf_nodes': [2, 4, 8, 16, 20, 22]}

g_search = GridSearchCV(DecisionTreeRegressor(), param_grid=param_grid, scoring='r2')

g_search.fit(x_train_final, y_train_final['IEMedia'].to_numpy())
print(f"Best score {g_search.best_score_} with {g_search.best_estimator_}")
best = g_search.best_estimator_

best.fit(x_train_final, y_train_final['IEMedia'])
y_pred = best.predict(x_test_final)
y_train_pred = best.predict(x_train_final)


print(f"R2 score para train {r2_score(y_train_final['IEMedia'], y_train_pred)}")
print(f"R2 score para test {r2_score(y_test_final['IEMedia'], y_pred)}")


Best score 0.6855982980945945 with DecisionTreeRegressor(max_depth=8, max_leaf_nodes=20)
R2 score para train 0.7649887652485289
R2 score para test 0.7015585393801441


Ahora que tenemos un modelo que podemos considerar bueno, podemos usar ese modelo para obtener, por ejemplo, la importancia de cada columna que ha encontrado el algoritmo.

In [14]:
feature_importances = pd.Series(data=clf.feature_importances_, index=x_train.columns)
print(feature_importances.sort_values(ascending=False)[:9])

SE5                    0.554482
SE6                    0.133386
Edad                   0.032329
EmpFut                 0.014038
CEF9                   0.012134
EmpFam                 0.012049
Active Saving (QF3)    0.010590
SEMedia                0.010282
Nota                   0.008718
dtype: float64


A partir de los resultados anteriores, podemos ver que variables influyen más en la intención emprendedora media:

- `SE5`: Conozco cómo desarrollar un proyecto empresarial.
- `SE6`: Si intentara iniciar una empresa, tendría una alta probabilidad de éxito.
- `Edad`: Edad del encuestado.
- `EmpFut`: Actualmente, el encuestado está tratando iniciar un negocio o ser autoempleado.
- `CEF9`: Preguntas sobre Conocimientos financieros empresariales
- `EmpFam`: Hay algún emprendedor en la familia del encuestado
- `Active Saving`: Ahorro activo.
- `SEMedia`: Variable global de Autoeficacia emprendedora
- `Nota`: Nota media del expediente 
